In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-130382")
exp = Experiment(workspace=ws, name="udacity_project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RTHU54T4A to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-130382
Azure region: southcentralus
Subscription id: 0c5a644d-c5ce-4e3b-bf42-4cb265317817
Resource group: aml-quickstarts-130382


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "cluster"

# Create a provisioning configuration using the Standard_D2_V2 VM and capping it at 4 nodes.
compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
# Create compute cluster.
cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0.1,10),
        "--max_iter": choice(20,30,40,50,60,70,80,90,100,120,140,160,180,190,200,300,400,600,800)
    }
)

# Specify a Policy
# If the current run has an absolute distance of > 0.2 from the current best run, terminate it. We check this every 10 iterations.
policy = BanditPolicy(evaluation_interval=1,
                      slack_amount=0.2,
                      delay_evaluation=10)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(".", compute_target=cpu_cluster, entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(ps,
                                     estimator=est,
                                     policy=policy,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100,
                                     max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails


hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [23]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

os.makedirs("outputs", exist_ok=True)

joblib.dump(value=best_run.id, filename="outputs/model.joblib")

['--C', '6.6291694063027915', '--max_iter', '160']


['outputs/model.joblib']

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 

url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=url_path)

In [6]:
# NOTE: Due to exceptions being thrown, I had to run my AutoML job on the remote cluster, which did not take the 'cleaned' pandas dataframe.
# Instead, it was required to use the TabularDataset and pass that to the AutoMLConfig.
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

# Split data into training and test data.
from sklearn.model_selection import train_test_split
import pandas as pd

x_train, x_test, y_train, y_test = train_test_split(x,y)

df_train = pd.concat([x_train,y_train],axis=1)

In [21]:
from azureml.train.automl import AutoMLConfig
import pandas as pd
# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    enable_onnx_compatible_models=True,
    compute_target=cpu_cluster,
    task="classification",
    primary_metric="accuracy",
    training_data=ds,
    validation_size=0.2,
    label_column_name="y",
    n_cross_validations=5)

In [22]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()

Running on remote.
No run_configuration provided, running on cluster with default configuration
Running on remote compute: cluster
Parent Run ID: AutoML_48d11475-4908-4653-b129-2eecdcce5549

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [40]:
# Retrieve and save your best automl model.
from azureml.automl.runtime.onnx_convert import OnnxConverter

best_automl, automl_model = automl_run.get_output()
best_automl, onnx_model = automl_run.get_output(return_onnx_model=True)
os.makedirs("outputs", exist_ok=True)

OnnxConverter.save_onnx_model(onnx_model,"./outputs/automl_model.onnx")

print(best_automl.get_tags())

{'_aml_system_azureml.automlComponent': 'AutoML', '_aml_system_ComputeTargetStatus': '{"AllocationState":"resizing","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":4}', 'ensembled_iterations': '[0, 1, 11, 14, 6, 9, 4]', 'ensembled_algorithms': "['LightGBM', 'XGBoostClassifier', 'XGBoostClassifier', 'XGBoostClassifier', 'XGBoostClassifier', 'LogisticRegression', 'RandomForest']", 'ensemble_weights': '[0.07142857142857142, 0.35714285714285715, 0.14285714285714285, 0.21428571428571427, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142]', 'best_individual_pipeline_score': '0.9180273141122914', 'best_individual_iteration': '0', '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True', 'model_explain_run_id': 'da1e1c5c-f076-4bb2-932f-def0ba872d8a', 'model_explanation': 'True', 'ModelExplainRunId': 'da1e1c5c-f076-4bb2-932f-def0ba872d8a'}


In [41]:
# Delete cluster at the end.

cpu_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

